In [16]:
import pandas as pd 
import numpy as np 
import random

In [2]:
from sklearn.datasets import load_diabetes

In [3]:
diabetes = load_diabetes()

In [4]:
X = diabetes.data
y = diabetes.target

### Firstly we calculate using the sklearn libraries.

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.metrics import r2_score

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2)

In [7]:
lr = LinearRegression()

In [8]:
lr.fit(X_train, y_train)

LinearRegression()

In [9]:
lr.coef_, lr.intercept_

(array([  -9.15865318, -205.45432163,  516.69374454,  340.61999905,
        -895.5520019 ,  561.22067904,  153.89310954,  126.73139688,
         861.12700152,   52.42112238]),
 np.float64(151.88331005254167))

In [10]:
r2_score(y_test, lr.predict(X_test))

0.4399338661568968

### Now we are create the own class that calculate cofficient and intercept using Mini-batch Gradient descent.

In [52]:
class MBGDRegressor: 

    def __init__(self, batch_size, learning_rate = 0.01, epochs = 100): 
        self.coef_ = None 
        self.intercept_ = None 
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size


    def fit(self, X_train, y_train): 
        
        # select the random value of intercept and cofficients. 
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])
        
        for epoch in range(self.epochs): 

            for batch in range(int(X_train.shape[0] / self.batch_size)): 
                # generate random index equal to batch size
                random_index = random.choices(range(0, X_train.shape[0]), k=self.batch_size)
                
                # update all the coef and the intercept
                y_hat = np.dot(X_train[random_index], self.coef_) + self.intercept_
                intercept_der = -2 * np.mean(y_train[random_index] - y_hat)
                self.intercept_ = self.intercept_ - (self.learning_rate * intercept_der)
    
                coef_der = -2 * np.dot((y_train[random_index] - y_hat), X_train[random_index]) / X_train[random_index].shape[0]
                self.coef_ = self.coef_ - (self.learning_rate * coef_der)

        print(self.intercept_, self.coef_)

    def predict(self, X_test): 
        return np.dot(X_test, self.coef_) + self.intercept_

In [67]:
mygd = MBGDRegressor(batch_size=30, learning_rate=0.2, epochs=100)

In [68]:
mygd.fit(X_train, y_train)

150.14510657825613 [ 4.22925428e+01 -8.24909642e+01  3.88248871e+02  2.67668999e+02
 -1.76231550e-01 -5.59683145e+01 -1.82902895e+02  1.25745378e+02
  3.54304461e+02  1.20636834e+02]


In [69]:
y_pred = mygd.predict(X_test) 

In [70]:
r2_score(y_test, y_pred)

0.44235726504015416

### Use the SGDRegressor built-in class inn sklearn.

In [103]:
from sklearn.linear_model import SGDRegressor

In [150]:
sgdlr = SGDRegressor(max_iter=70, learning_rate='constant', random_state=42)

In [157]:
# batch_size initialization. 
batch_size = 35
epochs = 50
for batch in range(epochs): 
    random_index = random.choices(range(0, X_train.shape[0]), k=batch_size)
    sgdlr.partial_fit(X_train[random_index], y_train[random_index])

In [158]:
sgdlr.intercept_, sgdlr.coef_

(array([135.612279]),
 array([ 36.48695839,   4.39765479, 102.24829621,  76.55590178,
         31.23171973,  22.08280124, -66.01922302,  66.34591045,
        103.48541303,  60.14134552]))

In [159]:
r2_score(y_test, sgdlr.predict(X_test))

0.1656097145400275